In [1]:
#!pip freeze

In [2]:
#pip install "dask[complete]"

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
#Inputs
input_file = 'online_retail_II.csv'
product_table= 'Product.csv' #dimensions need to exist before this is run

#Outputs
output_file = 'TransformedData.csv'
order_table = 'Orders.csv' #normailising more as date repetition is large
order_details_table = 'OrderDetails.csv'

In [5]:
# First Check
df = pd.read_csv(input_file)
df.head(25)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.0,United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.0,United Kingdom
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,13085.0,United Kingdom


In [6]:
df.describe(include='all')

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
count,1067371,1067371,1062989,1.067371e+06,1067371,1.067371e+06,824364.000000,1067371
unique,53628,5305,5698,NaN,47635,NaN,NaN,43
top,537434,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,2010-12-06 16:57:00,NaN,NaN,United Kingdom
freq,1350,5829,5918,NaN,1350,NaN,NaN,981330
mean,NaN,NaN,NaN,9.938898e+00,NaN,4.649388e+00,15324.638504,NaN
std,NaN,NaN,NaN,1.727058e+02,NaN,1.235531e+02,1697.464450,NaN
min,NaN,NaN,NaN,-8.099500e+04,NaN,-5.359436e+04,12346.000000,NaN
25%,NaN,NaN,NaN,1.000000e+00,NaN,1.250000e+00,13975.000000,NaN
50%,NaN,NaN,NaN,3.000000e+00,NaN,2.100000e+00,15255.000000,NaN
75%,NaN,NaN,NaN,1.000000e+01,NaN,4.150000e+00,16797.000000,NaN


In [7]:
# Add columns we may use for analysis here, some will not make final model
df['Revenue'] = df['Quantity'] * df['Price'] 
df['RevenueAbs'] = df['Revenue'].abs() #returns have negative quantity
df['ReturnFlag'] = df['Invoice'].apply(lambda x: 1 if x.startswith('C') else 0)
df['Customer ID'] = df['Customer ID'].apply(lambda customer_id: 0 if pd.isna(customer_id) or not customer_id else customer_id) #need these filled in
df['Description'] = df['Description'].apply(lambda desc: 'Blank' if pd.isna(desc) or not desc else desc) #need these filled in
df['Year'] = df['InvoiceDate'].str[:4].astype(int) #for graphing later on
df['Month'] = df['InvoiceDate'].str[5:7].astype(int)
df['MonthYear'] = df['InvoiceDate'].str[:7]
df['InvoiceDateTime'] = df['InvoiceDate']
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']).dt.date #this needs to be a date for min aggregation later on
df['Time'] = pd.to_datetime(df['InvoiceDateTime'].str[-8:], format='%H:%M:%S').dt.time #this needs to be a date for min aggregation later on
#df.sort_values(by=['InvoiceDate','RevenueAbs'],inplace=True,ascending=False)

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue,RevenueAbs,ReturnFlag,Year,Month,MonthYear,InvoiceDateTime,Time
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01,6.95,13085.0,United Kingdom,83.4,83.4,0,2009,12,2009-12,2009-12-01 07:45:00,07:45:00
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,81.0,81.0,0,2009,12,2009-12,2009-12-01 07:45:00,07:45:00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,81.0,81.0,0,2009,12,2009-12,2009-12-01 07:45:00,07:45:00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01,2.10,13085.0,United Kingdom,100.8,100.8,0,2009,12,2009-12,2009-12-01 07:45:00,07:45:00
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01,1.25,13085.0,United Kingdom,30.0,30.0,0,2009,12,2009-12,2009-12-01 07:45:00,07:45:00


In [8]:
df.describe(include='all')

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue,RevenueAbs,ReturnFlag,Year,Month,MonthYear,InvoiceDateTime,Time
count,1067371,1067371,1067371,1.067371e+06,1067371,1.067371e+06,1.067371e+06,1067371,1.067371e+06,1.067371e+06,1.067371e+06,1.067371e+06,1.067371e+06,1067371,1067371,1067371
unique,53628,5305,5699,NaN,604,NaN,NaN,43,NaN,NaN,NaN,NaN,NaN,25,47635,821
top,537434,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,2010-12-06,NaN,NaN,United Kingdom,NaN,NaN,NaN,NaN,NaN,2011-11,2010-12-06 16:57:00,14:09:00
freq,1350,5829,5918,NaN,7756,NaN,NaN,981330,NaN,NaN,NaN,NaN,NaN,84711,1350,3973
mean,NaN,NaN,NaN,9.938898e+00,NaN,4.649388e+00,1.183570e+04,NaN,1.806987e+01,2.122850e+01,1.826357e-02,2.010426e+03,7.496765e+00,NaN,NaN,NaN
std,NaN,NaN,NaN,1.727058e+02,NaN,1.235531e+02,6.596929e+03,NaN,2.924202e+02,2.922079e+02,1.339030e-01,5.737608e-01,3.526416e+00,NaN,NaN,NaN
min,NaN,NaN,NaN,-8.099500e+04,NaN,-5.359436e+04,0.000000e+00,NaN,-1.684696e+05,0.000000e+00,0.000000e+00,2.009000e+03,1.000000e+00,NaN,NaN,NaN
25%,NaN,NaN,NaN,1.000000e+00,NaN,1.250000e+00,1.253000e+04,NaN,3.750000e+00,3.750000e+00,0.000000e+00,2.010000e+03,5.000000e+00,NaN,NaN,NaN
50%,NaN,NaN,NaN,3.000000e+00,NaN,2.100000e+00,1.452500e+04,NaN,9.900000e+00,9.950000e+00,0.000000e+00,2.010000e+03,8.000000e+00,NaN,NaN,NaN
75%,NaN,NaN,NaN,1.000000e+01,NaN,4.150000e+00,1.637200e+04,NaN,1.770000e+01,1.770000e+01,0.000000e+00,2.011000e+03,1.100000e+01,NaN,NaN,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 16 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Invoice          1067371 non-null  object 
 1   StockCode        1067371 non-null  object 
 2   Description      1067371 non-null  object 
 3   Quantity         1067371 non-null  int64  
 4   InvoiceDate      1067371 non-null  object 
 5   Price            1067371 non-null  float64
 6   Customer ID      1067371 non-null  float64
 7   Country          1067371 non-null  object 
 8   Revenue          1067371 non-null  float64
 9   RevenueAbs       1067371 non-null  float64
 10  ReturnFlag       1067371 non-null  int64  
 11  Year             1067371 non-null  int32  
 12  Month            1067371 non-null  int32  
 13  MonthYear        1067371 non-null  object 
 14  InvoiceDateTime  1067371 non-null  object 
 15  Time             1067371 non-null  object 
dtypes: float64(4), int

In [10]:
df.memory_usage(deep=True) #each column is 68mb (value / 125k)

Index                   132
Invoice            58724905
StockCode          57757750
Description        80682054
Quantity            8538968
InvoiceDate        42694840
Price               8538968
Customer ID         8538968
Country            66618497
Revenue             8538968
RevenueAbs          8538968
ReturnFlag          8538968
Year                4269484
Month               4269484
MonthYear          59772776
InvoiceDateTime    72581228
Time               51233808
dtype: int64

In [11]:
df['Revenue'].agg(['sum','min','median','max']) #19.287m is revenue, needs to add up later on

sum       1.928725e+07
min      -1.684696e+05
median    9.900000e+00
max       1.684696e+05
Name: Revenue, dtype: float64

In [12]:
#Return Reason for Order Details
'''this piece came after a little manual excel analysis of the duplicates in Product dim, found some returns used description field. Want to keep that.
In the real world you'd have a historic product info table and use that to help determine if a description was a return reason
, this method is possible to error slightly and have dq issues as we have only current products we had to decide on'''

product = pd.read_csv(product_table)

#Use the product info to help assign return reason, may be useful much later on though I expect issues
df = df.merge(product[['StockCode', 'Description']], on=['StockCode'], how='left', suffixes=('_original', '_product')) #like SQL join
df['ReturnReason'] = np.where((df['ReturnFlag'] == 1) & (df['Description_original'] != df['Description_product']),df['Description_original'], '')

#at this point the original description column goes, in the model it'll be in the product lookup, in reality we expect the business to provide the info
#names should only change for a procuct if it's he same ID, would create new SKU for new product
df = df.drop(['Description_original', 'Description_product'], axis=1) 
#df.sort_values(by=['InvoiceDate','RevenueAbs'],inplace=True,ascending=False)
print(df[df['ReturnReason'] != ''].head())

     Invoice StockCode  Quantity InvoiceDate  Price  Customer ID    Country  \
179  C489449    85206A        -6  2009-12-01   1.65      16321.0  Australia   
180  C489449     21895        -4  2009-12-01   4.25      16321.0  Australia   
182  C489449     22083       -12  2009-12-01   2.95      16321.0  Australia   
184  C489449     84946       -12  2009-12-01   1.25      16321.0  Australia   
186  C489449     22090       -12  2009-12-01   2.95      16321.0  Australia   

     Revenue  RevenueAbs  ReturnFlag  Year  Month MonthYear  \
179     -9.9         9.9           1  2009     12   2009-12   
180    -17.0        17.0           1  2009     12   2009-12   
182    -35.4        35.4           1  2009     12   2009-12   
184    -15.0        15.0           1  2009     12   2009-12   
186    -35.4        35.4           1  2009     12   2009-12   

         InvoiceDateTime      Time                     ReturnReason  
179  2009-12-01 10:33:00  10:33:00     CREAM FELT EASTER EGG BASKET  
180  2

In [13]:
#check returns are as labelled, using business logic as return invoices begin with a 'C' there are clear issues
#should be investigated
chek = df.copy()
chek['PostiveRevenue'] = chek['Revenue'].apply(lambda x: 'Y' if x > 0 else 'N')
chek['ReturnFlag1'] = df['Invoice'].apply(lambda x: 1 if x.upper().startswith('C') else 0)
chek['ReturnFlag2'] = df['Invoice'].str.contains('C', case=False).astype(int)
print(chek[['ReturnFlag', 'PostiveRevenue','Revenue']].groupby(['ReturnFlag', 'PostiveRevenue']).agg(['count','sum','min','max']))

                           Revenue                                     
                             count           sum         min        max
ReturnFlag PostiveRevenue                                              
0          N                  6207 -1.586761e+05  -53594.360      -0.00
           Y               1041670  2.097259e+07       0.001  168469.60
1          N                 19493 -1.527041e+06 -168469.600      -0.12
           Y                     1  3.735700e+02     373.570     373.57


In [14]:
#case insensitive
print(chek[['ReturnFlag1', 'PostiveRevenue','Revenue']].groupby(['ReturnFlag1', 'PostiveRevenue']).agg(['count','sum','min','max']))

                            Revenue                                     
                              count           sum         min        max
ReturnFlag1 PostiveRevenue                                              
0           N                  6207 -1.586761e+05  -53594.360      -0.00
            Y               1041670  2.097259e+07       0.001  168469.60
1           N                 19493 -1.527041e+06 -168469.600      -0.12
            Y                     1  3.735700e+02     373.570     373.57


In [15]:
#anywhere case insensitive
print(chek[['ReturnFlag2', 'PostiveRevenue','Revenue']].groupby(['ReturnFlag2', 'PostiveRevenue']).agg(['count','sum','min','max']))

                            Revenue                                     
                              count           sum         min        max
ReturnFlag2 PostiveRevenue                                              
0           N                  6207 -1.586761e+05  -53594.360      -0.00
            Y               1041670  2.097259e+07       0.001  168469.60
1           N                 19493 -1.527041e+06 -168469.600      -0.12
            Y                     1  3.735700e+02     373.570     373.57


In [ ]:
#Main table for the model if not going with Order Details, drop columns we won't use
df.drop(['RevenueAbs','Year','Month','MonthYear'], axis=1).to_csv(output_file, index=False)
print(f"Data transformed and saved to: {output_file}")

Orders and Order Details work

Order Details

In [ ]:
#Option 2 for the model, we split into orders and have an order details section. This would be better data IMO
#note, could actually try to shorten country using country codes but will not do here

#Order Details table, will retain country and customer in order table as they're unique per order
#also will do some work to take latest InvoiceDate as the order date, 2 min diff is trivial
df.sort_values(by=['InvoiceDateTime', 'Invoice','RevenueAbs'],ascending=[False,True,False]).drop(
    ['RevenueAbs','Year','Month','PostiveRevenue','ReturnFlag'
         ,'Customer ID','Country' 
         ,'MonthYear','InvoiceDateTime','InvoiceDate','Time' 
        ], axis=1
       ).to_csv(order_details_table, index=False)
print(f"Data transformed and saved to: {order_details_table}")

In [ ]:
doublecheck=pd.read_csv(order_details_table, usecols=['Revenue'])
doublecheck['Revenue'].agg(['sum','min','median','max']) #19.287m is revenue, needs to add up later on

Orders

In [ ]:
#no issues here, will use the min order date time as it's when the order was started, 2 min differeence is negligible anyway
Orders = df.groupby(['Invoice','Customer ID','InvoiceDate','Country','ReturnFlag'])[['Time','Revenue','Quantity']].agg(
    Time=('Time', 'min'),
    Revenue=('Revenue', 'sum'),
    Quantity=('Quantity', 'sum')
)
Orders.reset_index().head()

In [ ]:
Orders['Revenue'].agg(['sum','min','median','max'])

In [ ]:
Orders.reset_index().sort_values(by=['InvoiceDate','Time', 'Invoice','Revenue'],ascending=[True,True,True,False]
                                ).to_csv(order_table, index=False)
print(f"Data transformed and saved to: {order_table}")

In [ ]:
#some visuals to help understand what we can present #note we know there are issues with group by so these are not for board level
monthlyRevenue = df[['MonthYear','Revenue']].groupby('MonthYear').agg(['sum'])
monthlyOrders = df[['MonthYear','Revenue']].groupby('MonthYear').agg(['count'])
#monthlyRevenue.set_index('MonthYear')
monthlyRevenue.head()

monthlyRevenue.plot(kind='bar', xlabel='Month', ylabel='Revenue £m', legend = False
                    ,title='Monthly Revenue', yticks=[200000,400000,600000,800000,1000000,1200000,1400000])

In [ ]:
#this one shows not a lot of year on year change, presentation should perhaps be more customer and product focussed?
filtered_data = df[df['Year'].astype(int) > 2009]
monthlyRevenue1 = filtered_data[['Month','Year','Revenue']].groupby(['Month','Year'])['Revenue'].sum().reset_index()
monthlyOrders1 = filtered_data[['Month','Year','Revenue']].groupby(['Month','Year']).agg(['count']).count().reset_index()
#monthlyRevenue1.info()
#data[data['year'] > 2009]

plt.figure(figsize=(18, 10))
sns.barplot(x='Month', y="Revenue",  hue = 'Year', data=monthlyRevenue1  ); 

In [ ]:
df.describe()

In [ ]:
Orders.reset_index().describe()